In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_data
import c3d_model
import math
import numpy as np
from tools import placeholder_inputs
from tools import _variable_with_weight_decay
from tools import tower_loss
from tools import tower_acc
from tools import _variable_on_cpu
from tools import _variable_with_weight_decay
from tools import average_gradients
from tools import get_logits
import input_data_for_extract_feature as input_data
# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1 , 'Batch size.')
FLAGS = flags.FLAGS
model_name = "../sports1m_finetuning_ucf101.model"
test_list_file = 'list/test.list'
unit_of_clip=2
data_dir = "./bin_data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)  
    
def run_test():
    num_test_videos = len(list(open(test_list_file,'r')))
    print("Number of test videos={}".format(num_test_videos))

    images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size * gpu_num)
    feature_placeholder, _ = placeholder_inputs(FLAGS.batch_size * gpu_num*unit_of_clip)

    fc6 = get_logits(images_placeholder, labels_placeholder,FLAGS,gpu_num)     
    saver = tf.train.Saver()
    
    
    tf_config = tf.ConfigProto()  
    tf_config.gpu_options.per_process_gpu_memory_fraction = 0.8
    sess = tf.Session(config=tf_config)

#     sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    init = tf.global_variables_initializer()######
    sess.run(init)
    saver.restore(sess, model_name)
    
    next_batch_start = 0
    all_steps = int((num_test_videos - 1) / (FLAGS.batch_size * gpu_num) + 1)
    
    
#     writer = tf.python_io.TFRecordWriter('fc6.tfrecord')
    file_index = 0
    fc6_batch_size=FLAGS.batch_size
    for step in xrange(all_steps):
#         try:
        np_arr_data, np_arr_label, next_batch_start, _, _ = input_data.read_clip_and_label(
                    test_list_file,FLAGS.batch_size,unit_of_clip,start_pos=next_batch_start)     
        fc6_of_batch=[]
        print("np_arr_data",np.shape(np_arr_data))
        batch_seq_arr = np_arr_data.reshape((fc6_batch_size * unit_of_clip, 16,112,112,3))
        print("batch_seq_arr",np.shape(batch_seq_arr))

        for batch_unit_index in range(0,unit_of_clip):
            temp_batch_data = batch_seq_arr[batch_unit_index*fc6_batch_size:batch_unit_index*fc6_batch_size + fc6_batch_size]
            batch_seq_fc6 = sess.run(fc6,feed_dict={images_placeholder:temp_batch_data})
            fc6_of_batch.append(batch_seq_fc6)
            #print("np.shape(batch_seq_fc6)",np.shape(batch_seq_fc6))
#         print("np.shape(fc6_of_batch)",np.shape(fc6_of_batch))
        fc6_of_batch = np.array(fc6_of_batch)
        fc6_feature_batch = fc6_of_batch.reshape((fc6_batch_size,unit_of_clip * 4096))
#         print("np.shape(fc6_feature_batch)",np.shape(fc6_feature_batch))
#         print("np_arr_label",np_arr_label)
        #print(fc6_feature_batch)

        for batch_index in range(fc6_batch_size):            
            try:
                #image = io.imread(images[i]) # type(image) must be array!
                data = fc6_feature_batch[batch_index]
                label = np_arr_label[batch_index]
                file_index += 1
                filename = "%s/%08d_%02d.bin" % (data_dir, file_index, label)
                print("data-->",data)
                print("label-->",label)
                print("filename",filename)
                with open(filename, 'wb') as f:
                    f.write(data)    
            except IOError as e:
                print('Skip it!\n')     
            
    print("Done!")

def main(_):
    run_test()

if __name__ == '__main__':
    tf.app.run()

Number of test videos=538
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/141...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 1.1347512  0.         0.        ...,  0.         0.         0.7203266]
label--> 0
filename ./bin_data/00000001_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/142...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.69827873  0.         ...,  0.          0.11517167
  0.61281729]
label--> 0
filename ./bin_data/00000002_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/143...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 2.35276651  0.          0.         ...,  0.11239719  0.          0.43851334]
label--> 0
filename ./bin_data/00000003_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/144...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.  0.  0. ...,  0.  0.  0.]
label--> 0
filename ./bin_data/00000031_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/172...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          1.21781659 ...,  0.          0.          0.61735481]
label--> 0
filename ./bin_data/00000032_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/173...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.  0.  0. ...,  0.  0.  0.]
label--> 0
filename ./bin_data/00000033_00.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/174...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          1.33287287 ...,  0.          0.          0.61445796]
label--> 0
filename ./bin_data/00000034_00.bin
Loading a video clip from /samba/HDD/dataset_

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          1.50861216]
label--> 1
filename ./bin_data/00000061_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/144...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.26706791  0.         ...,  0.          0.67472297
  0.34095573]
label--> 1
filename ./bin_data/00000062_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/145...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          1.62930632]
label--> 1
filename ./bin_data/00000063_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/146...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          1.22266173
  0.0354

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.  0.  0. ...,  0.  0.  0.]
label--> 1
filename ./bin_data/00000091_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/174...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.96755767 ...,  0.          0.37170923
  0.23090208]
label--> 1
filename ./bin_data/00000092_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/175...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.  0.  0. ...,  0.  0.  0.]
label--> 1
filename ./bin_data/00000093_01.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/176...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.38203055 ...,  0.          0.          0.49153298]
label--> 1
filename ./bin_data/00000094_01.bin
Loading a video clip from /samba/HDD/dataset

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          1.43331671]
label--> 2
filename ./bin_data/00000121_02.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/144...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.79378992  0.         ...,  0.          0.02438384
  0.43938911]
label--> 2
filename ./bin_data/00000122_02.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/145...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          1.81900597]
label--> 2
filename ./bin_data/00000123_02.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/146...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.69206637
  0.2524

...

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          1.09526479 ...,  0.          0.          0.        ]
label--> 7
filename ./bin_data/00000450_07.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/173...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          0.20099521]
label--> 7
filename ./bin_data/00000451_07.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/174...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          1.04878998 ...,  0.          0.          0.        ]
label--> 7
filename ./bin_data/00000452_07.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/175...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.          0.          0.534658

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.54104567 ...,  0.          0.          0.        ]
label--> 8
filename ./bin_data/00000480_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/142...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.93854415  0.          0.        ]
label--> 8
filename ./bin_data/00000481_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/143...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.54104567 ...,  0.          0.          0.        ]
label--> 8
filename ./bin_data/00000482_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/144...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.93854415  0.          0.      

np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.54104567 ...,  0.          0.          0.        ]
label--> 8
filename ./bin_data/00000510_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/172...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.93854415  0.          0.        ]
label--> 8
filename ./bin_data/00000511_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/173...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.54104567 ...,  0.          0.          0.        ]
label--> 8
filename ./bin_data/00000512_08.bin
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/174...
np_arr_data (1, 2, 16, 112, 112, 3)
batch_seq_arr (2, 16, 112, 112, 3)
data--> [ 0.          0.          0.         ...,  0.93854415  0.          0.      

SystemExit: 

/home/wjj/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
b = np.fromfile("./bin_data/00000001_00.bin", dtype=np.float32)  

array([ 1.1347512,  0.       ,  0.       , ...,  0.       ,  0.       ,
        0.7203266], dtype=float32)